In [20]:
from flask import Flask, render_template, request
import joblib
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

Copiert und modifiziert von https://medium.com/nerd-for-tech/simple-linear-regression-with-deployment-using-flask-cafbf85fa96f

In [21]:
X = np.array([10, 20, 30, 40, 50, 10, 25, 30, 45, 50, 15, 20, 35, 40, 50, 50, 45, 30, 25, 20, 10])
Y = np.array([95, 185, 280, 370, 490, 100, 230, 290, 410, 500, 135, 200, 295, 395, 495, 480, 430, 305, 205, 175, 110])

data = pd.DataFrame({'Ads/Month':X, 'Paid/Month':Y})
data.head()

,Ads/Month,Paid/Month
0,10,95
1,20,185
2,30,280
3,40,370
4,50,490


In [22]:
model = LinearRegression().fit(X.reshape(-1, 1), Y)

In [23]:
filename = "../webserver/model.sav"
#joblib.dump(model, "../webserver/"+str(filename))

In [24]:
from flask import Flask, render_template, request
import numpy as np
import joblib
import threading


In [25]:
app = Flask(__name__, template_folder="../webserver")

@app.route('/')
def student():
    return render_template("home.html")


def ValuePredictor(to_predict_list):
    to_predict = np.array(to_predict_list).reshape(-1, 1)
    loaded_model = joblib.load("../webserver/linear_model.pkl")
    result = loaded_model.predict(to_predict)
    return result[0]


@app.route('/', methods=["POST", "GET"])
def result():
    if request.method == "POST":
        to_predict_list = request.form.to_dict()
        to_predict_list = list(to_predict_list.values())
        to_predict_list = list(map(float, to_predict_list))
        result = round(float(ValuePredictor(to_predict_list)), 2)
        return render_template("home.html", result=result)

def run_app():
    app.run(port=5000, debug=True)

# Start Flask in a separate thread
threading.Thread(target=run_app).start()

In [15]:
import src.dataPipeline as dataPipeline
import importlib
importlib.reload(dataPipeline)

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import joblib
importlib.reload(dataPipeline)

<module 'src.dataPipeline' from 'C:\\Users\\wartm\\Documents\\FHNW\\immo_challenge\\src\\dataPipeline.py'>

In [16]:
dp = dataPipeline.DataPipeline()
df = dp.runPipeline(
    filePath="../data/immo_data_202208_v2.csv",
    imputer=None,
    normalizeAndStandardize= False,
    basic_house_imputer = True,
    web=True,
    get_dummies = False
)

C:\Users\wartm\Documents\FHNW\immo_challenge\src\dataPipeline.py:44: DtypeWarning: Columns (3,4,5,6,11,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,45,46,47,49,50,107,110,114,115,116,119,120,121,124,125,126,128,131,132) have mixed types. Specify dtype option on import or set low_memory=False.
  self.data = pd.read_csv(filePath)
C:\Users\wartm\Documents\FHNW\immo_challenge\src\dataPipeline.py:71: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ]].bfill(axis=1)['Space extracted']
C:\Users\wartm\Documents\FHNW\immo_challenge\src\dataPipeline.py:76: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in 

In [17]:
df.head()

,Availability,Floor,detail_responsive#surface_usable,Floor_space_merged,lat,lon,ForestDensityL,ForestDensityM,ForestDensityS,NoisePollutionRailwayL,...,gde_workers_total,price_cleaned,Space extracted,type_unified,Plot_area_unified,No. of rooms:,Last refurbishment:,Year built:,Number of floors:,region_group
0,On request,4.0,0.0,NaN,47.417100,8.085600,0.511176,0.286451,0.090908,0.0,...,331.0,1150000.0,100.0,penthouse,0.0,5.0,NaN,NaN,1.0,67.0
1,On request,0.0,0.0,242.0,47.419500,8.082700,0.511176,0.286451,0.090908,0.0,...,331.0,1420000.0,156.0,terrace-house,222.0,5.0,NaN,NaN,1.0,67.0
2,Immediately,2.0,0.0,NaN,47.403300,8.033000,0.163362,0.095877,0.001911,0.0,...,33493.0,720000.0,93.0,penthouse,0.0,5.0,NaN,NaN,1.0,67.0
3,On request,0.0,0.0,257.0,47.415643,8.085423,0.511176,0.286451,0.090908,0.0,...,331.0,1430000.0,154.0,detached-house,370.0,5.0,NaN,NaN,1.0,67.0
4,On request,0.0,0.0,NaN,47.403824,8.048288,0.333865,0.279276,0.145835,0.0,...,1355.0,995000.0,142.0,flat,0.0,5.0,NaN,NaN,1.0,67.0


In [18]:
df = df.drop(columns=["Availability","region_group"])

In [19]:
experiment_standort = ["Floor","price_cleaned","detail_responsive#surface_usable","Number of floors:","Plot_area_unified","Space extracted","No. of rooms:","type_unified", "lon","lat"]
df_3 = df[experiment_standort].copy()
df_3.head()

,Floor,price_cleaned,detail_responsive#surface_usable,Number of floors:,Plot_area_unified,Space extracted,No. of rooms:,type_unified,lon,lat
0,4.0,1150000.0,0.0,1.0,0.0,100.0,5.0,penthouse,8.085600,47.417100
1,0.0,1420000.0,0.0,1.0,222.0,156.0,5.0,terrace-house,8.082700,47.419500
2,2.0,720000.0,0.0,1.0,0.0,93.0,5.0,penthouse,8.033000,47.403300
3,0.0,1430000.0,0.0,1.0,370.0,154.0,5.0,detached-house,8.085423,47.415643
4,0.0,995000.0,0.0,1.0,0.0,142.0,5.0,flat,8.048288,47.403824


In [20]:
df_3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20809 entries, 0 to 22480
Data columns (total 10 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Floor                             17684 non-null  float64
 1   price_cleaned                     20809 non-null  float64
 2   detail_responsive#surface_usable  20809 non-null  float64
 3   Number of floors:                 20809 non-null  float64
 4   Plot_area_unified                 20809 non-null  float64
 5   Space extracted                   19923 non-null  float64
 6   No. of rooms:                     20185 non-null  float64
 7   type_unified                      20809 non-null  object 
 8   lon                               20809 non-null  float64
 9   lat                               20809 non-null  float64
dtypes: float64(9), object(1)
memory usage: 1.7+ MB


In [21]:
class KMeansClusterer(BaseEstimator, TransformerMixin):
    def __init__(self, n_clusters=50):
        self.n_clusters = n_clusters
        self.kmeans = KMeans(n_clusters=n_clusters, random_state=42)

    def fit(self, X, y=None):
        self.kmeans.fit(X)
        return self

    def transform(self, X):
        cluster_labels = self.kmeans.predict(X)
        return pd.DataFrame(cluster_labels, columns=["region_group"], index=X.index)

# Define features
numeric_features = [
    "Floor",
    "detail_responsive#surface_usable",
    "Number of floors:",
    "Plot_area_unified",
    "Space extracted",
    "No. of rooms:",
]
categorical_features = ["type_unified"]

# Step 1: KMeans clustering pipeline for `lon` and `lat`
kmeans_pipeline = Pipeline(steps=[
    ('kmeans', KMeansClusterer(n_clusters=50))
])

# Apply KMeans clustering to extract region groups
region_group = kmeans_pipeline.fit_transform(df_3[['lon', 'lat']])
df_3['region_group'] = region_group.values

# Step 2: Preprocessing pipeline
# Preprocessing pipeline for numeric features
numeric_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=5)),
    ('scaler', StandardScaler())
])

# Preprocessing pipeline for categorical features
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(drop='first'))
])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features + ['region_group'])
    ]
)

# Prepare data for training
X = df.drop(columns=["price_cleaned"])  # Features
y = np.log(df["price_cleaned"])         # Target variable (log-transformed)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

# Apply KMeans clustering to train and test sets
region_group_train = kmeans_pipeline.fit_transform(X_train[['lon', 'lat']])
region_group_test = kmeans_pipeline.transform(X_test[['lon', 'lat']])
X_train['region_group'] = region_group_train.values
X_test['region_group'] = region_group_test.values

# Preprocess the data
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)


In [23]:
model = LinearRegression()
model.fit(X_train, y_train)


joblib.dump(kmeans_pipeline, "../webserver/kmeans_clusterer.pkl")
joblib.dump(preprocessor, "../webserver/preprocessor.pkl")
joblib.dump(model, "../webserver/linear_model.pkl")

# Load and use in prediction
loaded_kmeans = joblib.load("../webserver/kmeans_clusterer.pkl")
loaded_preprocessor = joblib.load("../webserver/preprocessor.pkl")
loaded_model = joblib.load("../webserver/linear_model.pkl")

# Prepare input data
region_group2 = loaded_kmeans.transform(X[['lon', 'lat']])
X['region_group'] = region_group2.values
X_test_preprocessed = loaded_preprocessor.transform(X)
y_pred = loaded_model.predict(X_test_preprocessed)

# Reverse log transformation for readability
y_pred_exp = np.exp(y_pred)
print(y_pred_exp)

[ 933385.55593722  923594.19266872  928960.73107777 ... 1426432.40980211
 1214029.86790705 1594821.15751964]
